# 第十二讲：$k$-means算法、高斯混合模型及最大期望算法

现在我们开始进入机器学习课程新的一章——无监督学习算法的应用。

## $k$-means聚类算法（$k$-means clustering algorithm）

在聚类问题（clustering problem）中，对于给定的训练集$\left\{x^{(1)},\cdots,x^{(m)}\right\}$，并且希望对样本分组，并找到能够使样本“紧密聚集”在一起的簇（clusters）。这里的$x^{(i)}\in\mathbb{R}^n$像原来一样，但是没有$y^{(i)}$。因此，这是一个无监督学习问题。

$k$-means聚类算法的过程如下：

1. 随机的初始化**聚类质心（cluster centroids）**$\mu_1,\mu_2,\cdots,\mu_k\in\mathbb{R}^n$；
2. 重复直到收敛

  `{`
  
  * 对每一个样本$i$，令$c^{(i)}:=\mathrm{arg}\displaystyle\operatorname*{min}_j\left\lVert x^{(i)}-\mu_j\right\rVert^2$
  * 对每一个簇$j$，令$\mu_j:=\frac{\sum_{i=1}^m1\left\{c^{(i)}=j\right\}x^{(i)}}{\sum_{i=1}^m1\left\{c^{(i)}=j\right\}}$
      
  `}`

在这个算法中，参数$k$表示我们想要聚类的簇的数量，$\mu_j$表示我们当前对于簇质心位置的猜测。为了初始化簇质心（即算法第一步），我们通常随机选择$k$个训练样本，然后令$k$个质心等于这$k$个训练样本。（方法不唯一，其他初始化方法也是可以的。）

算法的内层循环重复这两步：(i)将每个训练样本$x^{(i)}$分配给距离它们最近的簇质心$\mu_j$，(ii)移动每个簇质心$\mu_j$到其下辖样本集合的中心。下图为$k$-means算法的简单演示：

<img src="./resource/chapter12_image01.png" width="600" alt="" align=center />

图中圆点为训练样本，叉号为簇质心，演示的过程为：(a)原数据集；(b)随机初始化的簇质心（在本例中并不是由某两个随机样本作为簇质心）；(c-f)演示了$k$-means算法的两个迭代，在每次迭代中，我们都将训练样本分配给距离它们最近的簇质心（图中将样本与对应的簇质心标记为同一颜色）；然后将每个簇质心移动到其下辖样本的中心。

那么，$k$-means算法一定收敛吗？在特定的场景下，答案是肯定的。我们先定义**失真函数（distortion function）**：

$$
J(c,\mu)=\sum_{i=1}^m\left\lVert x^{(i)}-\mu_{c^{(i)}}\right\rVert^2
$$

$J$为每个样本$x^{(i)}$到其簇质心$\mu_{c^{(i)}}$的距离的平方之和，可以证明$k$-means正是在$J$上的坐标下降过程（参考[第八讲](chapter08.ipynb)）。尤其是$k$-means算法的内循环实际上是在重复：保持$\mu$不变最小化$J$关于$c$的函数，然后再保持$c$不变最小化$J$关于$\mu$的函数。因此，$J$是单调递减的，它的函数值一定收敛。（通常这意味着$c$和$\mu$也会收敛。但在理论上，$k$-means算法的最终结果可能在少数簇之间振荡，比如少数几组$c$或/且$\mu$具有完全一样的$J$值。不过在实际操作中，这种现象几乎不会发生。）

失真函数$J$是一个非凸函数，所以对于$J$的坐标下降并不能保证其收敛于全局最小值，即$k$-means会被局部最小值影响。尽管如此，$k$-means算法通常都都能很好的完成聚类任务。不过，如果真的怕算法落入“不好的”局部最优中，一个可行的解决方案是为簇质心$\mu_j$赋上不同的初始化值，多运行几次，然后选出失真函数$J(c,\mu)$最小的一组聚类结果即可。

## 混合高斯模型（Mixtures of Gaussians）与最大期望算法（EM algorithm）

在这一节我们将讨论对聚类样本密度估计（density estimation）的最大期望（EM: Expectation-Maximization）。

假设有给定训练集$\left\{x^{(1)},\cdots,x^{(m)}\right\}$，因为现在研究无监督学习算法，所以训练样本没有对应的$y$标记。

我们希望使用一个联合分布$p\left(x^{(i)},z^{(i)}\right)=p\left(x^{(i)}\mid z^{(i)}\right)p\left(z^{(i)}\right)$（链式法则）对数据建模。这里的$z^{(i)}\sim\mathrm{Multinomial}(\phi)$（即$\phi_j\gt0,\ \sum_{j=1}^k\phi_j=1,\ \phi_j=p\left(z^{(i)}=j\right)$，当只有两个高斯分布时，$z^{(i)}$只有两种取值的可能，它是一个伯努利分布），并且有$x^{(i)}\mid z^{(i)}=j\sim\mathcal{N}\left(\mu_j,\varSigma_j\right)$。令$k$表示事件$z^{(i)}$可能取的值的总数，因此，我们的模型假设每个事件$x^{(i)}$均由事件“$z^{(i)}$随机的在$\{1,\cdots,k\}$做出选择”生成，而$x^{(i)}$可以表示为$k$个“由$z^{(i)}$确定的高斯分布”中的一个。这就是**混合高斯模型（mixture of Gaussians model）**。应注意$z^{(i)}$是**潜在（latent）**随机变量，这意味着它们是隐藏的、未知的，也正是这一点加大了我们估值难度。

模型的参数为$\phi,\mu,\varSigma$，为了估计这些参数，可以写出参数在给定数据上的对数似然估计：

$$
\begin{align}
\mathscr{l}\left(\phi,\mu,\varSigma\right)&=\sum_{i=1}^m\log p\left(x^{(i)};\phi,\mu,\varSigma\right)\\
&=\sum_{i=1}^m\log\sum_{z^{(i)}=1}^kp\left(x^{(i)}\mid z^{(i)};\mu,\varSigma\right)p\left(z^{(i)};\phi\right)
\end{align}
$$

然而，如果我们对式中每个参数求偏导并置为零，并尝试解出极值，会发现我们无法求出这些参数的最大似然估计的解析解。

随机变量$z^{(i)}$指出了$x^{(i)}$服从$k$个高斯分布中的哪一个。可以看出，如果我们能够确定$z^{(i)}$，那么这个最大似然问题就会好解很多，我们可以将似然函数写成：

$$
\mathscr{l}\left(\phi,\mu,\varSigma\right)=\sum_{i=1}^m\log p\left(x^{(i)}\mid z^{(i)};\mu,\varSigma\right)+\log p\left(z^{(i)};\phi\right)
$$

最大化上面这个关于$\phi,\mu,\varSigma$的式子就能求得各参数：

$$
\begin{align}
\phi_j&=\frac{1}{m}\sum_{i=1}^m1\left\{z^{(i)}=j\right\}\\
\mu_j&=\frac{\sum_{i=1}^m1\left\{z^{(i)}=j\right\}x^{(i)}}{\sum_{i=1}^m1\left\{z^{(i)}=j\right\}}\\
\varSigma_j&=\frac{\sum_{i=1}^m1\left\{z^{(i)}=j\right\}\left(x^{(i)}-\mu_j\right)\left(x^{(i)}-\mu_j\right)^T}{\sum_{i=1}^m1\left\{z^{(i)}=j\right\}}
\end{align}
$$

如果能确定$z^{(i)}$，则这里的参数估计与前面（[第五讲](chapter05.ipynb)）高斯判别分析模型中的参数估计几乎是一样的，只不过$z^{(i)}$代替了原来的类标记$y^{(i)}$。（式子里还有其它细节不同，在[问题集1](http://cs229.stanford.edu/materials/ps1.pdf)的高斯判别分析中我们一了解到：第一，不同问题集中$y^{(i)}\in\{-1,1\}$服从伯努利分布，我们已经将$z^{(i)}$一般化为多项分布；第二，不同于问题集中不同的$y$共用一个协方差矩阵，我们已经给每个高斯分布赋上了不同的协方差矩阵$\varSigma_j$。）

然而，在我们的密度估计问题中，$z^{(i)}$是未知的，我们该怎么办呢？

EM算法（[中文](https://zh.wikipedia.org/wiki/%E6%9C%80%E5%A4%A7%E6%9C%9F%E6%9C%9B%E7%AE%97%E6%B3%95)，[英文](https://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm)）是一种迭代算法，有两个主要步骤。对于上面的问题，E步骤中，算法尝试猜测每个$z^{(i)}$的值，在M步骤中算法会根据猜测更新模型。因为M步骤会假设E步骤中的猜测是正确的，所以有了缺失的$z^{(i)}$，最大化就很简单了。算法的流程：

重复直到收敛：`{`
  
  * (E步骤)对于每个$i,j$，令$w_j^{(i)}:=p\left(z^{(i)}=j\mid x^{(i)};\phi,\mu,\varSigma\right)$（样本$x^{(i)}$属于第$j$个高斯分布的概率）
  * (M步骤)更新模型中的参数：
$$
\begin{align}
\phi_j&=\frac{1}{m}\sum_{i=1}^mw_j^{(i)}\\
\mu_j&=\frac{\sum_{i=1}^mw_j^{(i)}x^{(i)}}{\sum_{i=1}^mw_j^{(i)}}\\
\varSigma_j&=\frac{\sum_{i=1}^mw_j^{(i)}\left(x^{(i)}-\mu_j\right)\left(x^{(i)}-\mu_j\right)^T}{\sum_{i=1}^mw_j^{(i)}}
\end{align}
$$
`}`

在E步骤中，我们使用$x^{(i)}$以及当前的参数值计算所有的$z^{(i)}$，即使用贝叶斯定理：

$$
p\left(z^{(i)}=j\mid x^{(i)};\phi,\mu,\varSigma\right)=\frac{p\left(x^{(i)}\mid z^{(i)}=j;\mu,\varSigma\right)p\left(z^{(i)}=j;\phi\right)}{\sum_{i=1}^kp\left(x^{(i)}\mid z^{(i)}=l;\mu,\varSigma\right)p\left(z^{(i)}=l;\phi\right)}
$$

* 式中的$p\left(x^{(i)}\mid z^{(i)}=j;\mu,\varSigma\right)$为某个以$\mu_j$为期望、$\varSigma_j$为协方差的高斯分布在$x^{(i)}$处的密度估计，即$\frac{1}{\sqrt{(2\pi)^k\left\lvert\varSigma_j\right\rvert}}\exp\left(-\frac{1}{2}\left(x^{(i)}-\mu_j\right)^T\varSigma^{-1}\left(x^{(i)}-\mu_j\right)\right)$；
* $p\left(z^{(i)}=j;\phi\right)$为$\phi_j$。

E步骤中求出的$w_j^{(i)}$表示我们对$z^{(i)}$的“软”猜测。（“软”意味着一种概率上的猜测，从$[0,1]$区间取值；相对的“硬”猜测意味着最佳的一次性猜测，是一种对值的猜测，比如从集合$\{0,1\}$或$\{1,\cdots,k\}$中取值。）

我们也应该对比M步骤中$z^{(i)}$已知时参数更新的式子，能够发现，只有式子里的由高斯分布中每个样本点求出的指示函数$1\left\{z^{(i)}=j\right\}$变为了$w_j^{(i)}$，其余部分同原来一样。

同样也可以用前面的$k$-means聚类算法做对比，不同之处在于$k$-means算法将样本“硬”分配给不同的簇，EM算法用$w_j^{(i)}$做“软”分配。相似之处在于，EM算法也存在落入局部最优的可能。所以，使用不同的初始参数多运行几次对EM算法同样适用。

显然，可以很自然的解释EM算法对于$z^{(i)}$的迭代猜测，但是这个点子来自哪里？我们能对算法属性做出证明吗，比如证明其收敛性？在下一节，我们将给出一个更加一般化的EM算法的描述，它会使我们轻易的将EM算法应用在不同的具有潜在随机变量的估值问题中，而且也能够证明算法的收敛性。

\left(\right)
\left\{\right\}
\begin{align}\end{align}